#### Module Summary

> This module reduces input features using the below set of checks on the data -
> 1. Drop features with > 95% missing values
> 2. Drop Degenrate features (only one value)
> 3. Drop features with > 90% correlation with other features
> 4. Drop features with high data drift (CSI > 25%)
>
> The module also performs an Exploratory Data Analysis and some basic data preparations like removing leading and trailing blanks, data type transformations, handling special values, missing value imputation etc.
>
> **Input Files**
>   1. Development Data (csv)
>   2. Validation Data (csv)
>
> **Output Files**
>   1. Development & Validation Data with reduced features (pickle)
>   2. Feature Reduction Summary (excel)
>   3. Model metadata - will be required for scoring (pickle)

<h4> 1.1 Import Libraries & Parameters and Initialise Output

In [1]:
# Import Libraries and Parameters
import logit_config as cfg
exec(open('module_imports.py').read())

In [2]:
# Delete All Output Files
out_flist = glob.glob(f"{outpath}/*.csv") + glob.glob(f"{outpath}/*.xlsx") + glob.glob(f"{outpath}/*.pickle") + glob.glob(f"{outpath}/*.aes") + glob.glob(f"{outpath}/**/*.png", recursive=True) + glob.glob(f"{outpath}/*.html")
for f in out_flist:
    os.remove(f)
    
# Create Graph Folder
if not(os.path.exists(f"{outpath}/graph")):
    os.mkdir(f"{outpath}/graph")
    
# Initialise Output Files
create_empty_excel_template(f'{outpath}/{feature_reduction_outfile}', ['Drop Reason Summary', 'Missing %', 'Correlation', 'Data Drift', 'Information Value', 'RFE', 'Stepwise Logistic'])

# Initialise list of Variables to retaine
retain_var_list = id_varlist + [resp_var]

<h4> 1.2 Data Import and Pre-processing

In [3]:
# Import Data
dev_data = pd.read_csv(f"{fpath}/{dev_fname}", thousands=',')
val_data = pd.read_csv(f"{fpath}/{val_fname}", thousands=',')

# Data Consistency Check
char_col_list = [x for x in dev_data.select_dtypes(exclude=np.number).columns if x not in non_pred_varlist+drop_varlist]
val_extra_col_df = check_data_consistency(dev_data, val_data, char_col_list)
val_extra_col_df.to_csv(f"{outpath}/val_extra_val_mapping.csv", index=False)

Development & Validation data have the same columns


<font color=blue>The above section will export a csv file @ **.out/val_extra_val_mapping.csv**\
This lists out all categorical variables with 'new' values in validation dataset, that do not appear in the development data. All these values will be made missing values in the next step. Please make changes to the above file if necessary, before running the replace new values in validation data module.</font>

In [4]:
# Replace New Classes (classes that appear in val data but not in dev data) of categorical variables in Validation Data
val_extra_col_df = pd.read_csv(f"{outpath}/val_extra_val_mapping.csv")
val_remap_dict = {col: {r.Values: r['Replace Value'] for i, r in val_extra_col_df[val_extra_col_df['Variable'] == col].iterrows()} for col in val_extra_col_df['Variable'].unique()}
val_data = val_data.replace(val_remap_dict)

In [5]:
# Drop Unnecessary Variables and convert variable types
dev_data = pre_process_data(dev_data, drop_varlist)
val_data = pre_process_data(val_data, drop_varlist)

print(f"Development Data Shape: {dev_data.shape}\nDevelopment Response Rate: {np.round(dev_data[resp_var].mean()*100, 2)}%")
try:
    print(f"Validation Data Shape: {val_data.shape}\nValidation Response Rate: {np.round(val_data[resp_var].mean()*100, 2)}%")
except:
    pass

# Copy dtype of Dev Data to Validation Data
dev_dtypes_dict = dev_data.dtypes.to_dict()
val_data = val_data.astype({key: dev_dtypes_dict[key] for key in val_data.columns})

Development Data Shape: (341, 32)
Development Response Rate: 36.66%
Validation Data Shape: (228, 32)
Validation Response Rate: 38.16%


<h4> 1.3 Missing Value & Degenrate Value Check

In [6]:
dev_data, val_data, nmiss_data, drop_reason_data = nmiss_nunique_check(dev_data, val_data)
print(f"Updated Development Data Shape: {dev_data.shape}")

export_data_to_excel(nmiss_data, f'{outpath}/{feature_reduction_outfile}', sheetName='Missing %', pct_col_list=['dev_miss_pct', 'val_missing_pct'])

Missing Value cut-off being used: 0.95. Variables dropped: 0
Degenerate Variables (Variables having only one value) Dropped: 0
Updated Development Data Shape: (341, 32)


<h4> 1.4 Recursive Feature Elimination

In [ ]:
exec_RFE_ind = 0

In [ ]:
%%time

if execc_RFE_ind == 1:
    
    # Compute RFE List
    varlist = [x for x in dev_data.columns if x not in non_pred_varlist]
    RFE_varlist = get_RFE_features(dev_data.copy(), varlist, resp_var, n_features=50)
    retain_var_list = retain_var_list + RFE_varlist

    # Export Data
    RFE_df = pd.DataFrame(RFE_varlist, columns=['feature'])
    export_data_to_excel(RFE_df, feature_reduction_outfile, sheetName='RFE')

<h4> 1.5 Correlation Check

In [7]:
%%time

# Check Correlation
corr_check_df = corr_check(dev_data.drop(id_varlist, axis=1), resp_var, corr_cutoff)
corr_check_df['corr_val'] = corr_check_df['corr_val'].apply(lambda x: np.round(x, 4))

# Drop Features with high correlation
corr_drop_varlist = [x for x in corr_check_df['del_var'].tolist() if x not in retain_var_list]
dev_data.drop(corr_drop_varlist, axis=1, inplace=True)
print(f"Correlation cut-off being used: {corr_cutoff}\nVariables Dropped: {len(corr_drop_varlist)}\nUpdated Development Data Shape: {dev_data.shape}")

# Export Variable Dropout Reason
export_data_to_excel(corr_check_df, f'{outpath}/{feature_reduction_outfile}', sheetName='Correlation', pct_col_list='corr_val')
corr_check_df['drop_reason'] = 'high_correlation'
drop_reason_data = drop_reason_data.append(corr_check_df[['del_var', 'drop_reason']].rename(columns={'del_var': 'feature'}))

Correlation cut-off being used: 0.9
Variables Dropped: 22
Updated Development Data Shape: (341, 21)
CPU times: user 119 ms, sys: 12.8 ms, total: 132 ms
Wall time: 162 ms


<h4> 1.6 EDA, Missing Value Imputation

In [8]:
%%time

# Exploratory Analysis
profile = ProfileReport(dev_data, title="Pandas Profiling Report", explorative=True)
profile.to_file(f'{outpath}/eda_pandas_profiling.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 1min 48s, sys: 6.54 s, total: 1min 55s
Wall time: 2min 4s


In [9]:
# Exploratory Analysis
EDA_df = get_exploratory_analysis(dev_data, [x for x in dev_data.columns if x not in non_pred_varlist])
EDA_df.fillna('').to_csv(f'{outpath}/exploratory_analysis.csv', index=False)

In [10]:
# Create Default Missing Value Imputation Input Data
dtypes_df = create_mv_impute_input(dev_data.drop(non_pred_varlist, axis=1))
dtypes_df.to_csv(f'{outpath}/{mv_input}', index=False)

<font color=blue>The above section exports a csv file @ **.out/missing_value_imputation_input.csv**\
This will be used as an input to the missing value imputation module in the below section. Please review and make changes to the above csv file, if necessary, before running the missing value imputation module below.</font>

In [11]:
# Missing Value Imputation
imp_df = pd.read_csv(f'{outpath}/{mv_input}')
dev_data = impute_missing_values(dev_data, imp_df)
val_data = impute_missing_values(val_data, imp_df)

<h4> 1.6 Data Drift Check

In [12]:
# Compute CSI
csi_df = compute_csi(dev_data, val_data, resp_var, id_varlist)
csi_drop_var_df = csi_df[csi_df['CSI'] >= csi_cutoff]

# Drop Variables
csi_drop_varlist = [x for x in csi_drop_var_df['feature'].tolist() if x not in retain_var_list]
dev_data.drop(csi_drop_varlist, axis=1, inplace=True)
print(f"CSI cut-off being used: {csi_cutoff}\nVariables Dropped: {len(csi_drop_varlist)}\nUpdated Development Data Shape: {dev_data.shape}")

# Export Variable Dropout Reason
csi_drop_var_df['drop_reason'] = 'high_data_drift'
drop_reason_data = drop_reason_data.append(csi_drop_var_df[['feature', 'drop_reason']])
export_data_to_excel(csi_df, f'{outpath}/{feature_reduction_outfile}', sheetName='Data Drift')

CSI cut-off being used: 0.25
Variables Dropped: 1
Updated Development Data Shape: (341, 20)


<h4> 1.7 Data Export

In [13]:
# Feature Reduction Summary
export_data_to_excel(drop_reason_data, f'{outpath}/{feature_reduction_outfile}', sheetName='Drop Reason Summary')

# Save Updated Dev & Val Data
with open(f"{outpath}/model_data.pickle", 'wb') as f:
    pickle.dump({'dev_data_out1': dev_data, 'val_data_out1': val_data[dev_data.columns.tolist()]}, f)
    
# Save Model Metadata
with open(f"{outpath}/model_metadata.pickle", 'wb') as f:
    pickle.dump({'dev_dtypes_dict': dev_dtypes_dict, 'imp_df': imp_df}, f)